In [2]:
# !pip install numpy matplotlib pandas tensorflow scikit-learn scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras    
from sklearn.model_selection import train_test_split
import scipy.io 
import os

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = 'D:/Development/Dataset/Bone_Fracture/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(512,512)
)
validation_ds = keras.utils.image_dataset_from_directory(
    directory = 'D:/Development/Dataset/Bone_Fracture/val',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(512,512)
)
test_ds = keras.utils.image_dataset_from_directory(
    directory = 'D:/Development/Dataset/Bone_Fracture/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(512,512)
)

In [ ]:
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)
test_ds=validation_ds.map(process)

In [ ]:
from keras.models import Model
from keras.layers import *

In [ ]:
x= Input(shape=(512,512,3))

In [ ]:
from tensorflow.keras.layers import Dense, BatchNormalization, MaxPool2D, Flatten, Conv2D, Input, Add, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

def resnet_block(x, filters, kernel_size=3, strides=1, downsample=False):
    shortcut = x
    conv = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    bnorm = BatchNormalization()(conv)
    relu = ReLU()(bnorm)
    conv = Conv2D(filters, kernel_size, padding='same')(relu)
    bnorm = BatchNormalization()(conv)
    
    if downsample:
        shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same')(x)
        shortcut = BatchNormalization()(shortcut)
    
    add = Add()([shortcut, bnorm])
    output = ReLU()(add)
    return output

input_shape = (224, 224, 3)
x = Input(shape=input_shape)

# Initial Conv Layer
conv1 = Conv2D(64, (7, 7), strides=2, padding='same', activation='relu')(x)
bNorm1 = BatchNormalization()(conv1)
maxpool1 = MaxPool2D((3, 3), strides=2)(bNorm1)

# Adding ResNet Blocks
res_block1 = resnet_block(maxpool1, 64)
res_block2 = resnet_block(res_block1, 64)
res_block3 = resnet_block(res_block2, 128, strides=2, downsample=True)  # Downsample
res_block4 = resnet_block(res_block3, 128)

# Global Average Pooling and Output
gap = GlobalAveragePooling2D()(res_block4)
output = Dense(1000, activation='softmax')(gap)  # Assuming 1000 classes

model = Model(inputs=x, outputs=output)

model.summary()


In [ ]:
from keras.utils import plot_model
plot_model(model,show_shapes=True)